In [0]:
%pip install folium
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from pyspark.sql.functions import month, year, avg, when, col


spark = SparkSession.builder.getOrCreate()
silver_db = "airbnb_lab3.airbnb_silver"

table_la_listings = f"{silver_db}.silver_listings_la"
table_ny_listings = f"{silver_db}.silver_listings_nyc"
table_la_calendar = f"{silver_db}.silver_calendar_la"
table_ny_calendar = f"{silver_db}.silver_calendar_nyc"
table_la_reviews = f"{silver_db}.silver_reviews_la"
table_ny_reviews = f"{silver_db}.silver_reviews_nyc"

df_la_listings = spark.table(table_la_listings).withColumn("city", F.lit("Los Angeles"))
df_ny_listings = spark.table(table_ny_listings).withColumn("city", F.lit("New York"))
df_la_calendar = spark.table(table_la_calendar).withColumn("city", F.lit("Los Angeles"))
df_ny_calendar = spark.table(table_ny_calendar).withColumn("city", F.lit("New York"))
df_la_reviews = spark.table(table_la_reviews).withColumn("city", F.lit("Los Angeles"))
df_ny_reviews = spark.table(table_ny_reviews).withColumn("city", F.lit("New York"))

listings_all = df_la_listings.unionByName(df_ny_listings)
calendar_all = df_la_calendar.unionByName(df_ny_calendar)
reviews_all = df_la_reviews.unionByName(df_ny_reviews)

calendar_agg = calendar_all.groupBy("listing_id").agg(
    F.avg(F.col("price")).alias("avg_calendar_price"),
    F.count(F.when(F.col("is_occupied") == True, 1)).alias("occupied_days"),
    F.count("*").alias("total_calendar_days")
).withColumn(
    "occupancy_rate", F.col("occupied_days") / F.col("total_calendar_days")
)

reviews_agg = reviews_all.groupBy("listing_id").agg(
    F.count("*").alias("review_count_agg"),
    F.min("review_date").alias("first_review_date"),
    F.max("review_date").alias("last_review_date")
)

df_spark_all = listings_all.join(
    calendar_agg, "listing_id", "left"
).join(
    reviews_agg, "listing_id", "left"
)

df_analysis = df_spark_all.toPandas()

In [0]:
df_analysis['avg_calendar_price'] = df_analysis['avg_calendar_price'].fillna(df_analysis['price'])
df_analysis['occupancy_rate'] = df_analysis['occupancy_rate'].fillna(0)
df_analysis['review_count_agg'] = df_analysis['review_count_agg'].fillna(0)
df_analysis['number_of_reviews'] = df_analysis['number_of_reviews'].fillna(0)

In [0]:
display(df_analysis.head(20))

In [0]:
df_time_series_spark = calendar_all.withColumn(
    "month", F.month(F.col("date"))
).withColumn(
    "year", F.year(F.col("date"))
).groupBy(
    "city", "year", "month"
).agg(
    F.avg(F.when(F.col("is_available") == True, F.col("price"))).alias("avg_price_available"),
    F.avg(F.when(F.col("is_occupied") == True, 1).otherwise(0)).alias("occupancy_rate")
).orderBy(
    "city", "year", "month"
)

df_time_series_monthly = df_time_series_spark.toPandas()
display(df_time_series_monthly)

In [0]:
df_neighborhood_spark = df_spark_all.groupBy(
    "city", "neighbourhood"
).agg(
    F.avg("avg_calendar_price").alias("avg_price"),
    F.avg("occupancy_rate").alias("avg_occupancy"),
    F.avg("review_scores_rating").alias("avg_rating"),
    F.count("listing_id").alias("listing_count")
)

df_neighborhood = df_neighborhood_spark.toPandas()

display(df_neighborhood)

In [0]:
df_day_of_week_spark = calendar_all.withColumn(
    "day_of_week_str", F.date_format(F.col("date"), "E")
).groupBy(
    "city", "day_of_week_str"
).agg(
    F.avg(F.when(F.col("is_available") == True, F.col("price"))).alias("avg_price"),
    F.avg(F.when(F.col("is_occupied") == True, 1).otherwise(0)).alias("avg_occupancy")
)

df_day_of_week = df_day_of_week_spark.toPandas()

day_order = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
df_day_of_week['day_of_week_str'] = pd.Categorical(df_day_of_week['day_of_week_str'], categories=day_order, ordered=True)
df_day_of_week = df_day_of_week.sort_values(['city', 'day_of_week_str'])

display(df_day_of_week.head(14))

In [0]:
sns.set_theme(style="whitegrid")

print("--- Аналіз цін за днем тижня ---")
plt.figure(figsize=(14, 7))
sns.barplot(data=df_day_of_week, x='day_of_week_str', y='avg_price', hue='city')
plt.title('Середня ціна за днем тижня', fontsize=16)
plt.xlabel('День тижня')
plt.ylabel('Середня ціна (USD)')
plt.show()

print("\n--- Аналіз тижневої заповненості ---")
plt.figure(figsize=(14, 7))
sns.barplot(data=df_day_of_week, x='day_of_week_str', y='avg_occupancy', hue='city')
plt.title('Середня заповненість (Occupancy) за днем тижня', fontsize=16)
plt.xlabel('День тижня')
plt.ylabel('Середня заповненість')
plt.show()


print("\n--- Аналіз Топ-10 найдорожчих районів (з >20 оголошеннями) ---")
df_neighborhood_filtered = df_neighborhood[df_neighborhood['listing_count'] > 20]

fig, axes = plt.subplots(1, 2, figsize=(16, 8))
fig.suptitle('Топ-10 Найдорожчих Районів (за середньою ціною)', fontsize=20)

la_top_10 = df_neighborhood_filtered[df_neighborhood_filtered['city'] == 'Los Angeles'].nlargest(10, 'avg_price')
sns.barplot(ax=axes[0], data=la_top_10, y='neighbourhood', x='avg_price', palette='Reds_r')
axes[0].set_title('Los Angeles')
axes[0].set_xlabel('Середня ціна (USD)')
axes[0].set_ylabel('Район')

ny_top_10 = df_neighborhood_filtered[df_neighborhood_filtered['city'] == 'New York'].nlargest(10, 'avg_price')
sns.barplot(ax=axes[1], data=ny_top_10, y='neighbourhood', x='avg_price', palette='Blues_r')
axes[1].set_title('New York')
axes[1].set_xlabel('Середня ціна (USD)')
axes[1].set_ylabel('')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [0]:
print("--- Розподіл за типом кімнати ---")
plt.figure(figsize=(12, 7))
sns.countplot(data=df_analysis, x='room_type', hue='city', order=df_analysis['room_type'].value_counts().index)
plt.title('Розподіл типів житла (Room Type) в LA та NY', fontsize=16)
plt.xlabel('Тип житла')
plt.ylabel('Кількість оголошень')
plt.show()

In [0]:
price_cap = df_analysis['avg_calendar_price'].quantile(0.99)
df_analysis_clean = df_analysis[
    (df_analysis['avg_calendar_price'] > 0) & 
    (df_analysis['avg_calendar_price'] <= price_cap) &
    (df_analysis['avg_calendar_price'].notna())
]
df_analysis_clean['log_price'] = np.log1p(df_analysis_clean['avg_calendar_price'])

print(f"--- Розподіл Log-Price (до ${price_cap:.0f}) ---")
plt.figure(figsize=(12, 6))
sns.histplot(df_analysis_clean, x='log_price', hue='city', kde=True, bins=50)
plt.title('Розподіл Log-Price (з Календаря) для LA та NY', fontsize=16)
plt.xlabel('Log(Avg. Calendar Price + 1)')
plt.ylabel('Кількість оголошень')
plt.show()

In [0]:
print("--- Розподіл Заповненості (Occupancy Rate) ---")
plt.figure(figsize=(12, 6))
sns.histplot(df_analysis_clean, x='occupancy_rate', hue='city', kde=True, bins=40, binrange=(0,1))
plt.title('Розподіл Заповненості (Occupancy Rate)', fontsize=16)
plt.xlabel('Рівень заповненості (0.0 - 1.0)')
plt.ylabel('Кількість оголошень')
plt.show()

In [0]:
print("--- Місячні Тренди (Сезонність) ---")
fig, axes = plt.subplots(2, 1, figsize=(14, 12), sharex=True)
fig.suptitle('Місячні Тренди: Ціна та Заповненість', fontsize=18)

sns.lineplot(ax=axes[0], data=df_time_series_monthly, x='month', y='avg_price_available', hue='city', lw=3)
axes[0].set_title('Середня ціна (для доступних)')
axes[0].set_ylabel('Середня ціна (USD)')
axes[0].set_xticks(range(1, 13))

sns.lineplot(ax=axes[1], data=df_time_series_monthly, x='month', y='occupancy_rate', hue='city', lw=3)
axes[1].set_title('Середня Заповненість')
axes[1].set_ylabel('Заповненість (%)')
axes[1].set_xlabel('Місяць')

plt.tight_layout(rect=[0, 0.03, 1, 0.97])
plt.show()

print("\n--- Тижневі Тренди (Ціни вихідного дня) ---")
plt.figure(figsize=(14, 7))
sns.barplot(data=df_day_of_week, x='day_of_week_str', y='avg_price', hue='city')
plt.title('Середня ціна за днем тижня', fontsize=16)
plt.xlabel('День тижня')
plt.ylabel('Середня ціна (USD)')
plt.show()

In [0]:
print("--- Топ-10 Районів за Середньою Ціною (з >20 оголошеннями) ---")
df_neighborhood_filtered = df_neighborhood[df_neighborhood['listing_count'] > 20]

fig, axes = plt.subplots(1, 2, figsize=(16, 8))
fig.suptitle('Топ-10 Найдорожчих Районів (за середньою ціною)', fontsize=20)

la_top_price = df_neighborhood_filtered[df_neighborhood_filtered['city'] == 'Los Angeles'].nlargest(10, 'avg_price')
sns.barplot(ax=axes[0], data=la_top_price, y='neighbourhood', x='avg_price', palette='Reds_r')
axes[0].set_title('Los Angeles')
axes[0].set_xlabel('Середня ціна (USD)')
axes[0].set_ylabel('Район')

ny_top_price = df_neighborhood_filtered[df_neighborhood_filtered['city'] == 'New York'].nlargest(10, 'avg_price')
sns.barplot(ax=axes[1], data=ny_top_price, y='neighbourhood', x='avg_price', palette='Blues_r')
axes[1].set_title('New York')
axes[1].set_xlabel('Середня ціна (USD)')
axes[1].set_ylabel('')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


print("\n--- Топ-10 Районів за Заповненістю (з >20 оголошеннями) ---")
fig, axes = plt.subplots(1, 2, figsize=(16, 8))
fig.suptitle('Топ-10 Районів з Найвищою Заповненістю', fontsize=20)

la_top_occ = df_neighborhood_filtered[df_neighborhood_filtered['city'] == 'Los Angeles'].nlargest(10, 'avg_occupancy')
sns.barplot(ax=axes[0], data=la_top_occ, y='neighbourhood', x='avg_occupancy', palette='Greens_r')
axes[0].set_title('Los Angeles')
axes[0].set_xlabel('Середня заповненість')
axes[0].set_ylabel('Район')

ny_top_occ = df_neighborhood_filtered[df_neighborhood_filtered['city'] == 'New York'].nlargest(10, 'avg_occupancy')
sns.barplot(ax=axes[1], data=ny_top_occ, y='neighbourhood', x='avg_occupancy', palette='Oranges_r')
axes[1].set_title('New York')
axes[1].set_xlabel('Середня заповненість')
axes[1].set_ylabel('')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [0]:
print("--- Розподіл Рейтингів (для оголошень з > 0 відгуками) ---")
df_with_reviews = df_analysis_clean[df_analysis_clean['number_of_reviews'] > 0]

plt.figure(figsize=(12, 6))
sns.histplot(df_with_reviews, x='review_scores_rating', hue='city', kde=True, bins=30, binrange=(1,5))
plt.title('Розподіл Рейтингів (Якість оголошень)', fontsize=16)
plt.xlabel('Рейтинг (1-5)')
plt.ylabel('Кількість оголошень')
plt.show()

print("\n--- Залежність Рейтингу від Заповненості (Вибірка) ---")
df_sample = df_with_reviews.sample(frac=0.1)
sns.lmplot(
    data=df_sample, 
    x='review_scores_rating', 
    y='occupancy_rate', 
    hue='city', 
    height=6, 
    aspect=1.5, 
    scatter_kws={'alpha':0.3}
)
plt.title('Залежність Заповненості від Рейтингу', fontsize=16)
plt.xlabel('Рейтинг (1-5)')
plt.ylabel('Заповненість')
plt.show()

In [0]:
import folium
import branca.colormap as cm
from IPython.display import display

print("--- Інтерактивні карти цін і заповненості ---")

# Вибірка для відображення
df_map_sample = df_analysis_clean.sample(n=5000, random_state=42)
df_la_sample = df_map_sample[df_map_sample['city'] == 'Los Angeles']
df_ny_sample = df_map_sample[df_map_sample['city'] == 'New York']

# --- 1️⃣ КАРТИ ЦІН ---
price_scale = cm.linear.YlOrRd_09.scale(
    vmin=df_analysis_clean['avg_calendar_price'].min(), 
    vmax=df_analysis_clean['avg_calendar_price'].quantile(0.95)
)

print("\n1️⃣ Карта цін Los Angeles")
la_price_map = folium.Map(
    location=[34.0522, -118.2437], 
    zoom_start=10,
    tiles='CartoDB positron'  # однотонний світлий фон
)
for _, row in df_la_sample.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color=price_scale(row['avg_calendar_price']),
        fill=True,
        fill_opacity=0.8,
        popup=f"${row['avg_calendar_price']:.0f}"
    ).add_to(la_price_map)
display(la_price_map)

print("\n1️⃣ Карта цін New York")
ny_price_map = folium.Map(
    location=[40.7128, -74.0060], 
    zoom_start=10,
    tiles='CartoDB positron'  # однотонний світлий фон
)
for _, row in df_ny_sample.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color=price_scale(row['avg_calendar_price']),
        fill=True,
        fill_opacity=0.8,
        popup=f"${row['avg_calendar_price']:.0f}"
    ).add_to(ny_price_map)
display(ny_price_map)

In [0]:
occ_scale = cm.linear.YlOrRd_09.scale(
    vmin=df_analysis_clean['occupancy_rate'].min(), 
    vmax=df_analysis_clean['occupancy_rate'].max()
)

print("\n2️⃣ Карта заповненості Los Angeles")
la_occ_map = folium.Map(
    location=[34.0522, -118.2437], 
    zoom_start=10,
    tiles='CartoDB positron'
)
for _, row in df_la_sample.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color=occ_scale(row['occupancy_rate']),
        fill=True,
        fill_opacity=0.8,
        popup=f"Occupancy: {row['occupancy_rate']:.0%}"
    ).add_to(la_occ_map)
display(la_occ_map)

print("\n2️⃣ Карта заповненості New York")
ny_occ_map = folium.Map(
    location=[40.7128, -74.0060], 
    zoom_start=10,
    tiles='CartoDB positron'
)
for _, row in df_ny_sample.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        color=occ_scale(row['occupancy_rate']),
        fill=True,
        fill_opacity=0.8,
        popup=f"Occupancy: {row['occupancy_rate']:.0%}"
    ).add_to(ny_occ_map)
display(ny_occ_map)


In [0]:
print("--- Аналіз Хостів (Host Analysis) ---")

host_counts = df_analysis_clean.groupby('host_id')['listing_id'].count()
df_analysis_clean['host_listing_count'] = df_analysis_clean['host_id'].map(host_counts)

def categorize_host(count):
    if count == 1:
        return '1 Listing'
    elif 2 <= count <= 5:
        return '2-5 Listings'
    elif 6 <= count <= 10:
        return '6-10 Listings'
    elif 11 <= count <= 50:
        return '11-50 Listings'
    else:
        return '50+ Listings (Mega-Host)'

df_hosts = df_analysis_clean.drop_duplicates(subset=['host_id']).copy()
df_hosts['host_category'] = df_hosts['host_listing_count'].apply(categorize_host)

category_order = ['1 Listing', '2-5 Listings', '6-10 Listings', '11-50 Listings', '50+ Listings (Mega-Host)']

plt.figure(figsize=(14, 7))
sns.countplot(data=df_hosts, x='host_category', hue='city', order=category_order)
plt.title('Розподіл Хостів за кількістю оголошень', fontsize=16)
plt.xlabel('Категорія Хоста')
plt.ylabel('Кількість Унікальних Хостів')
plt.show()